In [1]:
# scripts/compare_proposals.py
import json
import glob
import os

def score_proposal(extracted):
    score = 0
    reasons = []
    # ejemplo: cada sección presente suma puntos
    if extracted.get("requisitos_tecnicos"):
        score += 40
    else:
        reasons.append("Falta requisitos técnicos")

    if extracted.get("condiciones_economicas"):
        # si incluye monto, suma más
        ce = extracted.get("condiciones_economicas", "")
        if "USD" in ce or any(ch.isdigit() for ch in ce):
            score += 30
        else:
            reasons.append("Condiciones económicas no detalladas")
    if extracted.get("clausulas_legales"):
        score += 30
    else:
        reasons.append("Falta cláusulas legales")
    # normalizar a 0-100
    score = min(score, 100)
    return score, reasons

def semaforo(score):
    if score >= 75:
        return "VERDE"
    if score >= 50:
        return "AMARILLO"
    return "ROJO"

# cargar todas las propuestas JSON (ejemplo)
proposals = {}
for path in glob.glob("processed/propuestas_estructuradas/*.json"):
    with open(path, "r", encoding="utf-8") as f:
        proposals[path] = json.load(f)

results = {}
for path, data in proposals.items():
    sc, reasons = score_proposal(data)
    results[os.path.basename(path)] = {"score": sc, "semaforo": semaforo(sc), "reasons": reasons}

# guardar resumen
os.makedirs("processed", exist_ok=True)
with open("processed/summary_comparison.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("Resumen comparativo guardado en processed/summary_comparison.json")

Resumen comparativo guardado en processed/summary_comparison.json
